In [57]:
import numpy as np
import pandas as pd
import graphlab
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

BASE  = './bytecup2016data'
IINFO = BASE + '/invited_info_train.txt'
VAL   = BASE + '/validate_nolabel.txt'
TEST   = BASE + '/test_nolabel.txt'

invdata = pd.read_csv(IINFO, delim_whitespace=True, header=None, names=["qid", "uid", "label"])
valdata = pd.read_csv(VAL)

In [58]:
def algorithm_predict(new_invdata, new_valdata):
    sf = graphlab.SFrame(new_invdata, format='dataframe')
    #m = graphlab.recommender.create(sf, target='rating')
    #m = graphlab.item_similarity_recommender.create(sf)
    #m = graphlab.factorization_recommender.create(sf,target='rating', num_factors=32, regularization=1e-9, 
                                                  #linear_regularization=1e-09, max_iterations=500)
    m = graphlab.recommender.ranking_factorization_recommender.create(sf, user_id='uid', item_id='qid', 
                                                                      target='label')
    #m = graphlab.popularity_recommender.create(sf, target='rating')  
    
    sf = graphlab.SFrame(new_valdata, format='dataframe')
    return m.predict(sf)

In [59]:
valdata['label'] = algorithm_predict(invdata, valdata)
valdata.to_csv('matrix_factorization-recommender.csv', separator=",")

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 245752 observations with 27127 users and 7708 items.

Data prepared in: 0.443309s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.097205

Final training RMSE: 0.230136

In [48]:
print "",str(sum([0.4151434821370934, 0.415209384158512, 0.41351925583323684, 0.40903968519099976, 0.4161257849170225]) / 5)
print "",str(sum([0.4164529878223203, 0.41688970440625606, 0.4148440425795822, 0.41031352673596194, 0.4176783775931639]) / 5)
print "",str(sum([0.41635797722212375, 0.4154277450022756, 0.4135618690187132, 0.4083892215575476, 0.4165047271638262]) / 5)

0.413807518447
0.415235727827
0.414048307993
